# 🤖 Formation : Construction d'une Application RAG PDF Chat

## Objectifs d'apprentissage
- Comprendre les concepts de RAG (Retrieval-Augmented Generation)
- Implémenter un système de recherche vectorielle avec FAISS
- Intégrer un modèle de langage local avec Ollama
- Créer une interface utilisateur avec Streamlit
- Gérer le traitement de documents PDF et la segmentation de texte

## Architecture du système
```
PDF → Chunking → Embeddings → Vector Store → Retrieval → LLM → Response
```

## 1. Installation et Imports

Commençons par installer les dépendances nécessaires et importer les modules requis.

In [ ]:
# Installation des dépendances (à exécuter une seule fois)
# !pip install streamlit faiss-cpu numpy PyPDF2 langchain scikit-learn ollama-python

from __future__ import annotations

import os
import tempfile
from typing import Any, Dict, List, Optional, Tuple

import faiss  # type: ignore
import numpy as np
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity

import ollama  # pip install ollama-python

## 2. Configuration de l'Interface Utilisateur

### 🎯 **Exercice 1** : Interface Streamlit
Complétez la configuration de l'interface utilisateur en ajoutant :
1. Le CSS pour styliser les messages de chat
2. Les styles pour les conteneurs de chat

In [ ]:
# Configuration de la page Streamlit
st.set_page_config(page_title="🤖 RAG PDF Chat", page_icon="🤖", layout="wide")

# TODO: Ajoutez le CSS pour styliser l'interface
# Indice: Utilisez st.markdown() avec unsafe_allow_html=True
# Créez des styles pour .user-msg et .bot-msg avec des couleurs différentes

st.markdown(
    """
<style>
html, body {
    font-family: 'Helvetica Neue', sans-serif;
    background-color: #f4f7fa;
}
.sidebar .sidebar-content {
    background-color: #ffffff;
}
.chat-container {
    max-width: 1200px;
    margin: auto;
    padding: 1rem;
}
/* TODO: Complétez les styles CSS pour les messages */
/* .user-msg { ... } */
/* .bot-msg { ... } */
</style>
""",
    unsafe_allow_html=True,
)

## 3. Configuration des Modèles et Paramètres

### Concepts clés :
- **Modèle de langage** : llama3.2:3b pour la génération de réponses
- **Modèle d'embeddings** : nomic-embed-text pour créer les représentations vectorielles
- **Paramètres de recherche** : TOP_K pour limiter les résultats

In [ ]:
# Configuration des modèles et paramètres
MODEL_NAME = "llama3.2:3b"
EMBEDDING_MODEL = "nomic-embed-text:latest"
DOC_TOP_K = 3
CHUNK_TOP_K = 5
CANDIDATES_K = 20
NEIGHBORS = 1
LAMBDA_DIVERSITY = 0.3
SIM_THRESHOLD = 0.25
TEMPERATURE = 0.2
MAX_TOKENS = 2048

## 4. Fonctions d'Interaction avec les Modèles

### 🎯 **Exercice 2** : Implémentation des appels LLM
Complétez les fonctions pour interagir avec Ollama :
1. `_call_llm()` : Appel au modèle de langage
2. `embed_texts()` : Génération d'embeddings

In [ ]:
def _call_llm(messages: List[Dict[str, str]], *, temperature: float = 0.1, max_tokens: int = 2048, stream: bool = False):
    """
    Fonction pour appeler le modèle de langage via Ollama
    
    Args:
        messages: Liste des messages au format [{"role": "user/assistant", "content": "..."}]
        temperature: Paramètre de créativité (0.0 = déterministe, 1.0 = créatif)
        max_tokens: Nombre maximum de tokens à générer
        stream: Si True, retourne un stream pour l'affichage en temps réel
    """
    # TODO: Implémentez l'appel à ollama.chat()
    # Indice: Utilisez ollama.chat() avec les paramètres model, messages, stream, options
    pass

def embed_texts(texts: List[str]) -> np.ndarray:
    """
    Génère les embeddings pour une liste de textes
    
    Args:
        texts: Liste des textes à vectoriser
        
    Returns:
        Array numpy contenant les embeddings
    """
    # TODO: Implémentez la génération d'embeddings
    # Indice: Utilisez ollama.embeddings() dans une liste comprehension
    # Retournez un np.array avec dtype="float32"
    pass

## 5. Traitement des Documents PDF

### Concepts techniques :
- **Extraction de texte** : PyPDF2 pour lire les PDFs
- **Nettoyage** : Expressions régulières pour nettoyer le texte
- **Normalisation** : Suppression des espaces et caractères indésirables

In [ ]:
def clean_text(text: str) -> str:
    """Nettoie le texte extrait du PDF"""
    import re
    text = re.sub(r"\n{2,}", "\n\n", text)  # Normalise les sauts de ligne
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)  # Répare les mots coupés
    text = re.sub(r"\s{2,}", " ", text)  # Normalise les espaces
    return text.strip()

def extract_pdf_text(path: str) -> str:
    """Extrait le texte d'un fichier PDF"""
    return clean_text("\n".join(page.extract_text() or "" for page in PdfReader(path).pages))

## 6. Segmentation de Documents (Chunking)

### 🎯 **Exercice 3** : Implémentation du chunking adaptatif
Le chunking est crucial pour le RAG. Implémentez :
1. `auto_chunk_size()` : Taille adaptée selon la longueur du document
2. `chunk_document()` : Segmentation du texte avec RecursiveCharacterTextSplitter

In [ ]:
def auto_chunk_size(tokens: int) -> int:
    """
    Détermine la taille optimale des chunks selon la longueur du document
    
    Args:
        tokens: Nombre approximatif de tokens dans le document
        
    Returns:
        Taille recommandée pour les chunks
    """
    # TODO: Implémentez la logique de taille adaptative
    # Règle: <8000 tokens → 1024, <20000 → 768, sinon 512
    pass

def chunk_document(text: str) -> List[str]:
    """
    Segmente un document en chunks avec chevauchement
    
    Args:
        text: Texte complet du document
        
    Returns:
        Liste des chunks de texte
    """
    size = auto_chunk_size(len(text.split()))
    
    # TODO: Créez un RecursiveCharacterTextSplitter
    # Paramètres: separators=["\n\n", "\n", ". "], chunk_size=size, 
    #            chunk_overlap=25% de size, length_function=len
    
    # TODO: Utilisez split_text() et filtrez les chunks < 100 caractères
    pass

## 7. Génération de Résumés

### Concept : Résumé hiérarchique
Pour les questions globales, nous générons un résumé du document complet qui sera utilisé comme contexte supplémentaire.

In [ ]:
def make_summary(text: str) -> str:
    """Génère un résumé structuré du document"""
    messages = [
        {
            "role": "system", 
            "content": "Vous êtes un expert en synthèse documentaire. Résumez le texte suivant en trois parties : (1) Contexte, (2) Points clés, (3) Conclusions. Répondez en français."
        },
        {"role": "user", "content": text[:120000]}  # Limite pour éviter de dépasser le contexte
    ]
    return _call_llm(messages)["message"]["content"].strip()

## 8. Index de Recherche Vectorielle

### 🎯 **Exercice 4** : Classe RagIndex
Implémentez la classe principale pour gérer l'indexation et la recherche :

### Architecture :
- **Index hiérarchique** : Documents → Chunks
- **FAISS** : Recherche vectorielle rapide
- **MMR** : Maximum Marginal Relevance pour la diversité

In [ ]:
class RagIndex:
    """Classe principale pour l'indexation et la recherche RAG"""
    
    def __init__(self):
        self.doc_index: Optional[faiss.IndexFlatIP] = None
        self.chunk_index: Optional[faiss.Index] = None
        self.doc_meta: List[Dict[str, Any]] = []
        self.chunk_meta: List[Dict[str, Any]] = []
        self.chunk_emb: Optional[np.ndarray] = None

    def build(self, uploaded_files: List[st.runtime.uploaded_file_manager.UploadedFile]):
        """
        Construit l'index à partir des fichiers uploadés
        
        Args:
            uploaded_files: Liste des fichiers PDF uploadés via Streamlit
        """
        doc_embs, chunk_embs_list = [], []
        
        for doc_id, uf in enumerate(uploaded_files):
            # Sauvegarde temporaire du fichier
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
                tmp.write(uf.getbuffer())
                path = tmp.name
            
            # TODO: Extraire le texte du PDF
            # TODO: Générer le résumé du document
            # TODO: Stocker les métadonnées du document
            # TODO: Créer l'embedding du résumé
            
            # TODO: Segmenter le texte en chunks
            # TODO: Générer les embeddings des chunks
            # TODO: Stocker les métadonnées des chunks
            
            os.unlink(path)  # Nettoyage

        # TODO: Créer l'index FAISS pour les documents (IndexFlatIP)
        # TODO: Créer l'index FAISS pour les chunks (IndexHNSWFlat)
        pass

## 9. Détection de Questions Globales

### Concept : Classification automatique des questions
Certaines questions portent sur l'ensemble du document (résumé, sujet principal) plutôt que sur des détails spécifiques.

In [ ]:
    def _is_global(self, query: str, thr: float = 0.78) -> bool:
        """
        Détermine si une question porte sur l'ensemble du document
        
        Args:
            query: Question de l'utilisateur
            thr: Seuil de similarité
            
        Returns:
            True si la question est globale
        """
        examples = [
            "De quoi parle ce document ?",
            "Quel est le sujet principal ?",
            "Fais un résumé du document",
        ]
        
        emb_q = embed_texts([query])[0]
        emb_ex = embed_texts(examples)
        
        # Calcul de similarité cosinus
        sims = emb_ex @ emb_q / (np.linalg.norm(emb_ex, axis=1) * np.linalg.norm(emb_q) + 1e-6)
        return float(np.max(sims)) >= thr

# Ajout à la classe RagIndex
RagIndex._is_global = _is_global

## 10. Maximum Marginal Relevance (MMR)

### 🎯 **Exercice 5** : Algorithme MMR
Implémentez l'algorithme MMR pour équilibrer pertinence et diversité dans les résultats.

### Formule MMR :
`Score = λ × Sim(query, doc) - (1-λ) × max(Sim(doc, selected))`

In [ ]:
    def _mmr(self, q: np.ndarray, cand: np.ndarray, k: int) -> List[int]:
        """
        Maximum Marginal Relevance pour diversifier les résultats
        
        Args:
            q: Embedding de la question
            cand: Embeddings des candidats
            k: Nombre de résultats à sélectionner
            
        Returns:
            Indices des candidats sélectionnés
        """
        selected, rest = [], list(range(len(cand)))
        
        while len(selected) < min(k, len(rest)):
            best, best_score = None, -1e9
            
            for idx in rest:
                # TODO: Calculer la similarité avec la question
                # sim_q = cosine_similarity(q, cand[idx])
                
                # TODO: Calculer la similarité maximale avec les documents déjà sélectionnés
                # sim_s = max(cosine_similarity(cand[idx], selected)) if selected else 0
                
                # TODO: Calculer le score MMR
                # score = LAMBDA_DIVERSITY * sim_q - (1-LAMBDA_DIVERSITY) * sim_s
                
                # TODO: Garder le meilleur candidat
                pass
            
            # TODO: Ajouter le meilleur à la sélection et le retirer des candidats
            
        return selected

# Ajout à la classe RagIndex
RagIndex._mmr = _mmr

## 11. Fonction de Recherche Principale

### 🎯 **Exercice 6** : Méthode retrieve()
Implémentez la logique de recherche complète :
1. Recherche des documents pertinents
2. Filtrage des chunks par document
3. Application de MMR
4. Expansion du contexte

In [ ]:
    def retrieve(self, query: str) -> Tuple[List[str], List[int], Optional[str]]:
        """
        Recherche les contextes pertinents pour une question
        
        Args:
            query: Question de l'utilisateur
            
        Returns:
            Tuple (contextes, indices, résumé_si_global)
        """
        # TODO: Générer l'embedding de la question
        
        # TODO: Rechercher les documents les plus pertinents (doc_index.search)
        
        # TODO: Filtrer les chunks appartenant aux documents sélectionnés
        
        # TODO: Créer un sous-index avec les chunks filtrés
        
        # TODO: Rechercher les chunks candidats
        
        # TODO: Appliquer le seuil de similarité
        
        # TODO: Appliquer MMR pour diversifier
        
        # TODO: Expansion du contexte (chunks voisins)
        
        # TODO: Construire les contextes finaux
        
        # TODO: Ajouter le résumé si question globale
        
        pass

# Ajout à la classe RagIndex
RagIndex.retrieve = retrieve

## 12. Construction des Prompts

### Concept : Ingénierie des prompts
Un prompt bien structuré améliore significativement la qualité des réponses du modèle.

In [ ]:
def build_prompt(question: str, contexts: List[str], summary: Optional[str], history: List[Dict[str, str]] = None):
    """
    Construit le prompt pour le modèle de langage
    
    Args:
        question: Question de l'utilisateur
        contexts: Contextes récupérés
        summary: Résumé du document si question globale
        history: Historique de conversation
        
    Returns:
        Messages formatés pour le LLM
    """
    # Construction du bloc de contexte avec numérotation
    ctx_block = "\n\n".join(f"[{i+1}] {c}" for i, c in enumerate(contexts))
    
    if summary:
        ctx_block = f"[Résumé] {summary}\n\n" + ctx_block
    
    system = (
        "Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. "
        "Citez vos sources avec les balises [n]. Si l'information n'est pas trouvée, informez-en l'utilisateur."
    )
    
    messages = [{"role": "system", "content": system}]
    
    # Ajout de l'historique de conversation
    if history and len(history) > 0:
        previous_messages = history[:-1] if history[-1]["role"] == "user" else history
        messages.extend(previous_messages)
    
    # Message utilisateur avec contexte
    current_user_content = f"CONTEXTE(S):\n{ctx_block}\n\nQUESTION: {question}\n\nRéponse:"
    messages.append({"role": "user", "content": current_user_content})
    
    return messages

## 13. Interface Streamlit Complète

### 🎯 **Exercice 7** : Application finale
Assemblons tous les composants pour créer l'application complète.

In [ ]:
# Initialisation de l'état Streamlit
if "messages" not in st.session_state:
    st.session_state.messages = []
if "rag" not in st.session_state:
    st.session_state.rag = None
if "processing" not in st.session_state:
    st.session_state.processing = False

# Sidebar pour l'upload de fichiers
with st.sidebar:
    st.header("📚 Documents")
    files = st.file_uploader("Déposez vos PDF", type=["pdf"], accept_multiple_files=True)
    if st.button("🔄 Réinitialiser"):
        st.session_state.clear()
        st.rerun()

# Construction de l'index
if files and st.session_state.rag is None:
    with st.spinner("📄 Indexation en cours…"):
        rag = RagIndex()
        rag.build(files)
        st.session_state.rag = rag
    st.success(f"{len(files)} document(s) indexé(s) ! Posez vos questions.")

## 14. Affichage du Chat et Gestion des Interactions

### 🎯 **Exercice 8** : Interface de chat complète
Implémentez l'affichage des messages et la gestion des interactions utilisateur.

In [ ]:
# TODO: Affichage des messages de chat
# Indice: Parcourir st.session_state.messages et afficher avec les styles CSS

# TODO: Gestion de l'input utilisateur
# Indice: Utiliser st.chat_input() avec les bonnes conditions de désactivation

# TODO: Traitement de la question
# 1. Ajouter le message utilisateur à l'historique
# 2. Récupérer les contextes avec rag.retrieve()
# 3. Construire le prompt
# 4. Appeler le LLM en streaming
# 5. Afficher la réponse en temps réel
# 6. Ajouter la réponse à l'historique

# TODO: Affichage des contextes dans un expander

## 15. Test de l'Application

### 🎯 **Exercice 9** : Tests et Validation
Une fois votre code complété, testez l'application avec différents types de questions :

1. **Questions spécifiques** : "Que dit le document sur [sujet précis] ?"
2. **Questions globales** : "De quoi parle ce document ?"
3. **Questions de comparaison** : "Quelles sont les différences entre X et Y ?"
4. **Questions factuelles** : "Quels sont les chiffres mentionnés ?"

In [ ]:
# Pour tester l'application, exécutez dans votre terminal :
# streamlit run formation_rag_chat.py

# Conseils de débogage :
print("✅ Configuration terminée")
print(f"📊 Modèle LLM : {MODEL_NAME}")
print(f"🔍 Modèle embeddings : {EMBEDDING_MODEL}")
print("🚀 Prêt pour les tests !")

## 16. Points d'Amélioration et Extensions

### Concepts avancés à explorer :

1. **Chunking sémantique** : Utiliser des modèles pour segmenter selon le sens
2. **Re-ranking** : Améliorer l'ordre des résultats avec des modèles dédiés  
3. **Mémoire conversationnelle** : Maintenir le contexte sur plusieurs échanges
4. **Multimodalité** : Supporter images et tableaux dans les PDFs
5. **Évaluation automatique** : Métriques RAGAS pour mesurer la qualité
6. **Cache intelligent** : Optimiser les performances avec mise en cache
7. **Agents RAG** : Ajouter des capacités de raisonnement et d'action

### Ressources pour aller plus loin :
- Documentation LangChain : https://python.langchain.com/
- Tutoriels FAISS : https://github.com/facebookresearch/faiss
- Guide Ollama : https://ollama.ai/

## 🎉 Félicitations !

Vous avez construit une application RAG complète capable de :
- ✅ Traiter des documents PDF
- ✅ Créer des embeddings vectoriels
- ✅ Effectuer une recherche sémantique
- ✅ Générer des réponses contextualisées
- ✅ Maintenir une interface utilisateur intuitive

**Prochaines étapes** : Déployez votre application et testez-la avec vos propres documents !